### import 부분

In [11]:
import numpy as np
import pandas as pd
from dataset import Dataset
from timeit import default_timer as timer

In [12]:
data = Dataset(data_path='Hana_Data.xlsx')
data.load_data(factory_sheet='수신_여신상품(상품팩토리)')
data.load_data(fund_sheet='펀드상품(펀드상품기본)')
data.load_data(bank_assurance='방카상품(방카슈랑스기본)')

## 수신 여신상품

In [13]:
code_name_dict = {}
error_index = []
correct_index = []
code_name = pd.concat([data.goods_factory['상품코드'], data.goods_factory['상품명']], axis=1)
for index, value in code_name.iterrows():
    if value['상품명'] not in code_name_dict.keys():
        code_name_dict[value['상품명']] = value['상품코드']
    else:
        if value['상품코드'] != code_name_dict[value['상품명']]:
            error_index.append([index, value['상품코드'], value['상품명']])
            correct_index.append([index, code_name_dict[value['상품명']]])



#### 상품코드와 상품명의 매핑 문제

##### 금전채권시탁과 여신성공탁금-여신성공탁금의 상품코드가 2개씩 매핑

In [14]:
pd.concat([pd.DataFrame(error_index,columns=["index",'상풍코드','상품명']),pd.DataFrame(correct_index,columns = ['index','상품코드'])], axis=1).iloc[[0,156],:]

,index,상풍코드,상품명,index,상품코드
0,151387,271028000101,금전채권신탁,151387,200041000101
156,221544,0380019W01301,여신성공탁금-여신성공탁금,221544,300088991001


#### 해결 - 금전채권시탁 : 271028000101 제거, 여신성공탁금-여신성공탁금 : 0380019W01301 제거 

In [15]:
data.goods_factory.drop(data.goods_factory.index[data.goods_factory['상품코드'] == 271028000101], inplace=True)
data.goods_factory.drop(data.goods_factory.index[data.goods_factory['상품코드'] == '0380019W01301'], inplace=True)

In [16]:
print(len(data.goods_factory['상품코드'].unique()))
print(len(data.goods_factory['상품명'].unique()))

1406
1406


#### 상품명과 판매시작일, 판매종료일은 잘 매핑 - 비어있으면 잘 중복 없이 잘 매핑

In [17]:
error = check_start_end_date()

In [18]:
error

[]

#### 조건코드와 조건설명의 매핑 및 중복 문제 - 비어있으면 정상

In [19]:
code_name = check_condition_code_explanation('조건코드', '조건설명', data.goods_factory)
name_code = check_condition_code_explanation('조건설명', '조건코드', data.goods_factory)

In [20]:
print(code_name)
print(name_code)

Empty DataFrame
Columns: [index, 조건코드_err, 조건설명_err, 조건코드_cor, 조건설명_cor]
Index: []
Empty DataFrame
Columns: [index, 조건설명_err, 조건코드_err, 조건설명_cor, 조건코드_cor]
Index: []


In [21]:
code_explain = check_condition_code_explanation(key_code='상세조건관리번호', value_word='상세조건설명', data_frame=data.goods_factory)

#### 상세조건관리번호 하나에 여러 상세조건설명이 Mapping

In [22]:
code_explain.head()

,index,상세조건관리번호_err,상세조건설명_err,상세조건관리번호_cor,상세조건설명_cor
0,150,A0160003,생계형저축한도-3000만원까지,A0160003,비과세종합저축한도-5000만원까지
1,275,A0160003,생계형저축한도-3000만원까지,A0160003,비과세종합저축한도-5000만원까지
2,423,A0160003,생계형저축한도-3000만원까지,A0160003,비과세종합저축한도-5000만원까지
3,611,A0160003,생계형저축한도-3000만원까지,A0160003,비과세종합저축한도-5000만원까지
4,785,A0160003,생계형저축한도-3000만원까지,A0160003,비과세종합저축한도-5000만원까지


##### 생계형저축한도-3000만원 관리코드를 A0160004로 변경

In [23]:
data.goods_factory.loc[data.goods_factory['상세조건설명'] == '생계형저축한도-3000만원까지', '상세조건관리번호'] = 'A0160004'

#### 상세조건설명에 여러 상세조건관리번호가 Mapping

In [24]:
explain_code = check_condition_code_explanation(key_code='상세조건설명', value_word='상세조건관리번호', data_frame=data.goods_factory)

In [25]:
explain_code.head()

,index,상세조건설명_err,상세조건관리번호_err,상세조건설명_cor,상세조건관리번호_cor
0,1374,하나은행 가족은 0.1% 우대금리를 드립니다,C0300008,하나은행 가족은 0.1% 우대금리를 드립니다,C0260008
1,1678,계약금액-100만원이상 2천만원이하,A00300DU,계약금액-100만원이상 2천만원이하,A00300JR
2,5505,계약금액-10만원이상 20만원이하,A00300JO,계약금액-10만원이상 20만원이하,A00300K5
3,5754,계약금액-10만원이상 20만원이하,A00300JO,계약금액-10만원이상 20만원이하,A00300K5
4,8919,10만원 이상 자동이체하시면 0.1% 우대금리를 드립니다,C0300010,10만원 이상 자동이체하시면 0.1% 우대금리를 드립니다,C0260010


#### 상세조건관리번호와 상제조건설명 수정코드

In [26]:
modify_detailed_code_explanation = []
data.goods_factory.groupby(by=['상세조건설명']).apply(mapping)
for key, value in detailed_condition_code_explanation_dict.items():
    detailed_name = value.pop()
    modify_detailed_code_explanation.append([key, detailed_name])
    data.goods_factory.loc[data.goods_factory['상세조건설명'] == key, '상세조건관리번호'] = detailed_name
modify_detailed_code_explanation = pd.DataFrame(modify_detailed_code_explanation, columns=['상세조건설명', '상세조건관리번호'])

#### 상세조건설명에 상세조건관리코드가 잘 매핑되어있는지 확인 - 비어있으면 잘 매핑

In [27]:
explain_code_check = check_condition_code_explanation(key_code='상세조건설명', value_word='상세조건관리번호', data_frame=data.goods_factory)

In [28]:
explain_code_check

,index,상세조건설명_err,상세조건관리번호_err,상세조건설명_cor,상세조건관리번호_cor


In [29]:
print(len(data.goods_factory['상세조건설명'].unique()))
print(len(data.goods_factory['상세조건관리번호'].unique()))

43538
43538


## 펀드상품

#### Nan 개수 중 '펀드투자성향구분'이 12124개로 삭제 

In [30]:
data.goods_fund.isnull().sum()

펀드번호               0
펀드상품명              0
운용사명              66
펀드종류명              0
운용형태구분명            0
수익증권운용형태중분류      716
펀드운용구분명            0
수익증권상품명            0
펀드위험수준구분         331
펀드투자지역구분          99
통화코드               0
감면과세상품구분명          0
추가설정구분             0
환매가능구분             0
수수료수취방법            0
환매수수료구간구분        701
마감후수용방법            0
매입체결일수             0
환매체결일수             0
환매결제일수             0
판매채널구분           442
투자목적구분           197
펀드투자성향구분       12124
투자기간구분           158
펀드공모사모구분           0
펀드포트폴리오유형        619
펀드투자등급유형         425
펀드시장전망유형        1545
펀드판매상품분류           0
펀드판매자격분류         661
dtype: int64

In [31]:
del data.goods_fund['펀드투자성향구분']

#### 하나의 펀드번호에 펀드상품명은 잘 매핑(중복 없음)

In [32]:
fund_num_goods = check_condition_code_explanation('펀드번호', '펀드상품명', data_frame=data.goods_fund)

In [33]:
fund_num_goods

,index,펀드번호_err,펀드상품명_err,펀드번호_cor,펀드상품명_cor


#### 하나의 펀드상품명에 여러 펀드번호가 매핑(중복 문제)

In [34]:
fund_goods_problem_code_explanation = check_condition_code_explanation('펀드상품명', '펀드번호', data_frame=data.goods_fund)
del fund_goods_problem_code_explanation['펀드상품명_cor']

In [35]:
fund_goods_problem_code_explanation.head()

,index,펀드상품명_err,펀드번호_err,펀드번호_cor
0,195,우리주가지수연동사모파생상품투자신탁116호,107106601046,107106301045
1,289,우리뉴인베스트사모혼합투자신탁제330호,107101401263,107101401256
2,555,하나UBSSafe4Chance2Y파생상품투자신탁제4호,102106101024,102106100960
3,564,산은H사모혼합투자신탁제29호,205101401260,205101401258
4,993,삼성배당프리미엄파생상품투자신탁제1호,105106100258,105106100257


#### 9개의 중복문제

In [36]:
print(len(fund_goods_problem_code_explanation['펀드상품명_err'].unique()))
print(len(fund_goods_problem_code_explanation['펀드상품명_err']))

54
63


In [37]:
fund_name_num_list = []
modify_frame = []
for index, value in fund_goods_problem_code_explanation.iterrows():
    if value['펀드상품명_err'] not in fund_name_num_list:
        # fund_goods_problem_code_explanation 중 중복 펀드상품명이 있기 때문에 이 중 하나만 처리
        fund_name_num_list.append(value['펀드상품명_err'])
        temp_frame = data.goods_fund.loc[data.goods_fund['펀드상품명'] == value['펀드상품명_err']]
        num_list = temp_frame['펀드번호'].tolist()
        modify_funds_num = num_list.pop()
        data.goods_fund.drop(data.goods_fund.index[data.goods_fund['펀드번호'].isin(num_list)], inplace=True)
        modify_frame.append([modify_funds_num, value['펀드상품명_err']])

In [38]:
fund_goods_modify_code_explanation = pd.DataFrame(modify_frame, columns=['펀드번호', '펀드상품명'])


#### 중복 제거하고 펀드상품명과 펀드번호 매핑정보보기

In [39]:
fund_goods_modify_code_explanation.head()

,펀드번호,펀드상품명
0,107106601046,우리주가지수연동사모파생상품투자신탁116호
1,107101401263,우리뉴인베스트사모혼합투자신탁제330호
2,102106101024,하나UBSSafe4Chance2Y파생상품투자신탁제4호
3,205101401260,산은H사모혼합투자신탁제29호
4,105106100258,삼성배당프리미엄파생상품투자신탁제1호


#### 모든 펀드상품이 Unique하다

In [40]:
print(len(data.goods_fund['펀드상품명']))
print(len(data.goods_fund['펀드상품명'].unique()))

15746
15746


#### 펀드번호와 펀드상품명이 잘 매핑(중복 문제해결)

In [41]:
print(len(data.goods_fund['펀드번호'].unique()))
print(len(data.goods_fund['펀드상품명'].unique()))

15746
15746


#### 데이터프레임이 비어있으므로 다시 한 번 잘 매핑됬는지 확인

In [42]:
frame = check_condition_code_explanation('펀드상품명', '펀드번호', data_frame = data.goods_fund)
del frame['펀드상품명_cor']
print(frame)

Empty DataFrame
Columns: [index, 펀드상품명_err, 펀드번호_err, 펀드번호_cor]
Index: []


## 방카상품 전처리

#### 결측치 개수 구하기

In [43]:
bank_assurance_null_count = data.bank_assurance_data.isnull().sum()

#### Nan 개수가 많으므로 우선 이를 모두 제거

In [44]:
bank_assurance_null_count[bank_assurance_null_count > 0]

예정이율            312
기준금액             77
최저보증이율          165
최저보증이율2         726
손보납입단위         2047
기간불확정코드        1455
연금개시최소연령        144
연금개시최대연령        118
적립여부            830
거치여부           1876
판매종료일자          148
최대펀드가입갯수       1560
특약적용CNT        2883
구좌가설구분         2986
배우자여부          2046
자녀구분            863
건강체구분          2090
우량체구분          2211
주계약담보여부        1850
보험료계산유형        2892
연금구분           1569
계약형태구분코드       4827
외화계좌사용여부       5324
비고             5332
판매제한내용         4801
보험료계산유형코드      3041
가족력입력여부        5326
거주자구분코드        3921
상품설명적용일자        679
적용기준가구분코드      4664
기준가적용일자        1330
보험사콜여부         3613
Unnamed: 45    5391
dtype: int64

### 제거

In [45]:
upper_3000 = bank_assurance_null_count[bank_assurance_null_count > 0]
for element in upper_3000.index:
    del data.bank_assurance_data[element]

#### 고려할 Column

In [46]:
print(data.bank_assurance_data.columns)

Index(['은행상품코드', '은행상품명', '보험사명', '가입목적', '상품보장기간', '저축보장구분', '보험사상품명', '원화구분',
       '남자최소가입연령', '남자최대가입연령', '여자최소가입연령', '여자최대가입연령', '판매시작일자'],
      dtype='object')


#### 은행상품코드와 은행상품명의 Mapping 문제

In [47]:
print(len(data.bank_assurance_data['은행상품코드'].unique()))
print(len(data.bank_assurance_data['은행상품명'].unique()))

5393
4136


In [48]:
error_frame = check_condition_code_explanation('은행상품명', '은행상품코드', data_frame=data.bank_assurance_data)
del error_frame['은행상품명_cor']

In [49]:
print(error_frame.head())
print(len(error_frame))

   index         은행상품명_err  은행상품코드_err  은행상품코드_cor
0     96     (무)베스트변액연금보험Ⅱ  L01184304K  L01184303K
1     98     (무)뉴베스트변액연금보험  L01184306K  L01184305K
2    128  연금저축하이드림Free연금보험  L01014203K  L01014202K
3    129  연금저축하이드림Free연금보험  L01014204K  L01014202K
4    130  연금저축하이드림Free연금보험  L01014205K  L01014202K
1257


#### 은행상품코드와 은행상품명 매핑 - (은행상품코드, 은행상품명, 보험사상품명)으로 매핑 

In [50]:
modify_code_bank_assurance = solve_bank_name_num_mapping(error_frame)

In [51]:
mapping_data = pd.DataFrame(modify_code_bank_assurance, columns=['은행상품코드', '은행상품명', '보험사상품명'])

In [52]:
mapping_data.head()

,은행상품코드,은행상품명,보험사상품명
0,L01184304K,(무)베스트변액연금보험Ⅱ,(무)베스트변액연금보험Ⅱ
1,L01184306K,(무)뉴베스트변액연금보험,(무)뉴베스트변액연금보험
2,L01014203K,연금저축하이드림Free연금보험,연금저축하이드림Free연금보험
3,L01014210K,연금저축스마트하이드림연금보험,연금저축스마트하이드림연금보험
4,L01024304K,(무)바로연금보험,(무)바로연금보험


#### (보험사상품명, 은행상품명)에 중복이 존재하지 않으면 Print되는 것이 없음

In [64]:
bank_assurance_check(data)

#### 띄어쓰기만 다르므로 삭제

In [54]:
data.bank_assurance_data.drop(data.bank_assurance_data.index[data.bank_assurance_data['은행상품명'] == '무배당교보First저축보험III'], inplace=True)

#### 저장

In [55]:
with pd.ExcelWriter('/home/kibum/recommender_system/Hana_Bank/Modify_Hana_Data.xlsx') as writer:
    data.goods_factory.to_excel(writer, sheet_name='수신_여신상품(상품팩토리)', index=False)
    data.goods_fund.to_excel(writer, sheet_name='펀드상품(펀드상품기본)', index=False)
    data.bank_assurance_data.to_excel(writer, sheet_name='방카상품(방카슈랑스기본)', index=False)

### 매핑(중복)검사시 Verify_Error.py 실행

## 관련함수

In [56]:
def check_start_end_date():
    """
    상품명과 판매시작일, 판매종료일이 잘 매핑되어있는지 확인 - 문제 없음
    """
    start_last_date_dict = {}
    error_index = []
    correct_index = []
    start_last_date = data.goods_factory[['상품명', '판매시작일자', '판매종료일자']]
    for index, value in start_last_date.iterrows():
        if value['상품명'] not in start_last_date_dict.keys():
            start_last_date_dict[value['상품명']] = [value['판매시작일자'], value['판매종료일자']]
        else:
            if value['판매시작일자'] != start_last_date_dict[value['상품명']][0] or value['판매종료일자'] != \
                    start_last_date_dict[value['상품명']][1]:
                error_index.append([index, value['상품명'], value['판매시작일자'], value['판매종료일자']])
                correct_index.append(
                    [index, start_last_date_dict[value['상품명']][0], start_last_date_dict[value['상품명']][1]])
    return error_index

In [57]:
def check_condition_code_explanation(key_code, value_word, data_frame):
    """
    매핑이 잘 되어있는지 확인하는 함수
    매핑이 1:1로 잘 되어있는 경우 데이터프레임에 아무 값도 없음
    매핑이 1:n으로 되어있는 경우 데이터프레임에 문제의 튜플이 담기게 됨
    """
    check_dict = {}
    error_index = []
    correct_index = []
    condition_code_explain = data_frame[[key_code, value_word]]
    # 여기서 조건코드와 조건설명을 바꿔서도 Check
    for index, line in condition_code_explain.iterrows():
        if line[key_code] not in check_dict.keys():
            check_dict[line[key_code]] = line[value_word]
        else:
            if line[value_word] != check_dict[line[key_code]]:
                error_index.append([index, line[key_code], line[value_word]])
                correct_index.append([line[key_code], check_dict[line[key_code]]])
    return pd.concat([pd.DataFrame(error_index, columns=['index', key_code + '_err', value_word + '_err']),
                      pd.DataFrame(correct_index, columns=[key_code + '_cor', value_word + '_cor'])], axis=1)



In [58]:
detailed_condition_code_explanation_dict = {}

In [59]:
def mapping(df):
    """
    람다 함수에 이용(apply)
    """
    detailed_condition_code_explanation_dict[df.name] = set(df['상세조건관리번호'].tolist())


In [60]:
def solve_bank_name_num_mapping(error_frame):
    """
    하나의 은행상품명에 여러 은행상품코드의 Mapping 문제 해결
    하나의 은행상품코드에는 하나의 은행상품명이 매핑되어있기 때문에 아래 코드에서 삭제 가능 가능
    """
    bank_code_name_dict = fill_bank_code_dict(error_frame)
    modify_code_bank_assurance = []
    for k, v in bank_code_name_dict.items():
        assurance_name = k[0]
        bank_goods_name = k[1]
        modify_key = v.pop()
        
        data.bank_assurance_data.drop(data.bank_assurance_data.index[data.bank_assurance_data['은행상품코드'].\
                                           isin(list(v))], inplace=True)
        modify_code_bank_assurance.append([modify_key, bank_goods_name, assurance_name])
    return modify_code_bank_assurance

In [61]:
def fill_bank_code_dict(error_frame):
    """
    방카상품에서 문제 있는 은행상품과 상품코드를 딕셔너리에 넣는 코드
    bank_code_name_dict : Key - (보험사상품명, 은행상품명), Value - 은행사상품코드 List
    """
    bank_code_name_dict = {}
    bank_name = []
    for index, value in error_frame.iterrows():
        if value['은행상품명_err'] not in bank_name:
            bank_name.append(value['은행상품명_err'])
            temp_frame = data.bank_assurance_data[data.bank_assurance_data['은행상품명'] == value['은행상품명_err']]
            temp_frame.groupby(by=['보험사상품명']).apply(lambda df: mappings(df, value['은행상품명_err'], bank_code_name_dict))
    return bank_code_name_dict

In [62]:
def mappings(df, bank_goods_name, bank_code_name_dict):
    """
    Lambda 함수를 위해 사용되는 코드
    """
    bank_code_name_dict[(df.name, bank_goods_name)] = set(df['은행상품코드'].tolist())



In [63]:
def bank_assurance_check(data):
    """
    Key : (은행상품명, 보험사상품명) Value : 은행상품코드
    error에서 출력이 되면 문제
    """
    bank_code_goodsname_assurancename = {}
    error = []
    for index, value in data.bank_assurance_data.iterrows():
        bank_key = (value['은행상품명'], value['보험사상품명'])
        if bank_key not in bank_code_goodsname_assurancename.keys():
            bank_code_goodsname_assurancename[bank_key] = value['은행상품코드']
        else:
            error.append(bank_key)
    if error:
        print(error)